# Teachable Machine 体験用サンプルコード

このノートブックでは、Teachable Machineで作成したモデルを使って画像認識を行います。

## 事前準備
1. Teachable Machine (https://teachablemachine.withgoogle.com/) でモデルを作成
2. モデルをエクスポート（Tensorflow → Keras形式）
3. `keras_model.h5`と`labels.txt`をダウンロード

In [ ]:
# 必要なライブラリのインストール
!pip install tensorflow pillow opencv-python numpy

In [ ]:
# Google Driveのマウント（モデルファイルを保存している場合）
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# ライブラリのインポート
import tensorflow as tf
import numpy as np
from PIL import Image
import cv2
from google.colab import files
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 日本語フォントの設定
!apt-get -y install fonts-ipafont-gothic
plt.rcParams['font.family'] = 'IPAGothic'

## モデルファイルのアップロード
Teachable Machineからダウンロードした`keras_model.h5`と`labels.txt`をアップロードしてください。

In [ ]:
# モデルファイルのアップロード
print("keras_model.h5 と labels.txt をアップロードしてください")
uploaded = files.upload()

In [ ]:
# モデルとラベルの読み込み
model = tf.keras.models.load_model('keras_model.h5', compile=False)

# ラベルの読み込み
with open('labels.txt', 'r', encoding='utf-8') as f:
    labels = [line.strip() for line in f.readlines()]

print("モデルの読み込み完了！")
print(f"認識できるクラス: {labels}")

## 画像認識関数の定義

In [ ]:
def predict_image(image_path):
    """
    画像を読み込んで予測を行う関数
    """
    # 画像の読み込みと前処理
    img = Image.open(image_path).convert('RGB')
    img = img.resize((224, 224))  # Teachable Machineは224x224を期待
    
    # numpy配列に変換して正規化
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    
    # 予測実行
    predictions = model.predict(img_array)
    
    # 結果を辞書形式で返す
    results = {}
    for i, label in enumerate(labels):
        # ラベルから番号を除去（例: "0 良品" → "良品"）
        clean_label = label.split(' ', 1)[1] if ' ' in label else label
        results[clean_label] = float(predictions[0][i])
    
    return results, img

def display_results(image_path):
    """
    画像と予測結果を表示する関数
    """
    results, img = predict_image(image_path)
    
    # 画像と結果を表示
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    # 画像表示
    ax1.imshow(img)
    ax1.axis('off')
    ax1.set_title('入力画像', fontsize=14)
    
    # 結果をグラフで表示
    labels_clean = list(results.keys())
    values = list(results.values())
    colors = ['green' if v == max(values) else 'gray' for v in values]
    
    bars = ax2.barh(labels_clean, values, color=colors)
    ax2.set_xlabel('確率', fontsize=12)
    ax2.set_title('予測結果', fontsize=14)
    ax2.set_xlim(0, 1)
    
    # 値をバーの上に表示
    for bar, value in zip(bars, values):
        ax2.text(value + 0.01, bar.get_y() + bar.get_height()/2, 
                f'{value:.1%}', va='center')
    
    plt.tight_layout()
    plt.show()
    
    # 最も確率の高いクラスを表示
    max_label = max(results, key=results.get)
    max_prob = results[max_label]
    print(f"\n予測結果: {max_label} (確率: {max_prob:.1%})")

## 画像のアップロードと認識

In [ ]:
# 画像をアップロード
print("認識したい画像をアップロードしてください")
uploaded = files.upload()

# アップロードされた画像を認識
for filename in uploaded.keys():
    print(f"\n=== {filename} の認識結果 ===")
    display_results(filename)

## カメラから画像を撮影して認識
Google Colabのカメラ機能を使って、その場で撮影した画像を認識します。

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import io

def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = '撮影';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
        f.write(binary)
    return filename

# カメラで撮影
print("カメラで画像を撮影します")
filename = take_photo()
print(f"撮影完了: {filename}")

# 撮影した画像を認識
display_results(filename)

## まとめ

このサンプルコードでは：
1. Teachable Machineで作成したモデルを読み込み
2. 画像をアップロードまたは撮影して認識
3. 結果をグラフで分かりやすく表示

次のステップ：
- より多くのデータで学習してモデルの精度を向上
- リアルタイムでの認識に挑戦
- 実際の製造現場での活用を検討